In [2]:
import cv2
import numpy as np
import pandas as pd
import easyocr #type:ignore
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

from glob import glob
from random import sample
from PIL import Image, ImageFont, ImageDraw, ImageEnhance

from pathlib import Path

In [ ]:
reader = easyocr.Reader(['en'], gpu=False)

In [ ]:
file = r"D:\jersey_images\2024-JUNE 13-MONDAY-TIME 1-FIELD 14-324-GAME-26471-TOURNAMENT-114.JPG"
reader.readtext(file, allowlist ='0123456789')

In [ ]:
img = Image.open(file)
img

In [ ]:
img = Image.open(file).resize((1280,1280))
img

In [ ]:
reader.readtext(np.array(img), allowlist ='0123456789')


In [ ]:
import cv2
import numpy as np
from paddleocr import PaddleOCR  

# Initialize the DBNet model
ocr = PaddleOCR(det_db_box_thresh=0.5, det_db_unclip_ratio=0.5, use_gpu=False)

# Preprocessing Function
def preprocess_image(image_path, target_size=(1280, 1280)):
    image = cv2.imread(image_path)
    original_image = image.copy()
    
    # Resize while maintaining aspect ratio
    h, w, _ = image.shape
    scale = min(target_size[0] / h, target_size[1] / w)
    new_h, new_w = int(h * scale), int(w * scale)
    image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
    
    # Add padding
    padded_image = np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)
    padded_image[:new_h, :new_w] = image

    return original_image, padded_image

# Draw bounding boxes with confidence
def draw_boxes(image, boxes):
    for box in boxes:
        points = np.array(box[0], dtype=np.int32).reshape((-1, 2))
        confidence = box[1]  # Confidence score
        if isinstance(confidence, list):  # Ensure confidence is a float
            confidence = confidence[0]
        # Draw the bounding box
        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 255), thickness=2)
        # Put the confidence score
        x, y = points[0]
        cv2.putText(image, f"{confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return image

# Crop and display detected regions
def crop_detected_regions(image, boxes):
    cropped_regions = []
    for idx, box in enumerate(boxes):
        points = np.array(box[0], dtype=np.int32).reshape((-1, 2))
        x_min = np.min(points[:, 0])
        y_min = np.min(points[:, 1])
        x_max = np.max(points[:, 0])
        y_max = np.max(points[:, 1])
        cropped_region = image[y_min:y_max, x_min:x_max]
        cropped_regions.append((idx, cropped_region))
    return cropped_regions

# Parameters
image_path = r"D:\jersey_testing\2024-JUNE 13-MONDAY-TIME 1-FIELD 1-324-GAME-26475-TOURNAMENT-88.JPG"
preprocessed_size = (640, 640)

# Preprocessing
original_image, preprocessed_image = preprocess_image(image_path, preprocessed_size)

# Text Detection
result = ocr.ocr(preprocessed_image, det=True, rec=False)

# Get bounding boxes and confidence
boxes_with_confidence = [(line[0], line[1]) for line in result[0]]
print(boxes_with_confidence)

# Visualize Results
image_with_boxes = draw_boxes(original_image.copy(), boxes_with_confidence)

# Crop Detected Regions
cropped_regions = crop_detected_regions(original_image.copy(), boxes_with_confidence)

# Save and Display Results
cv2.imshow("Original Image", original_image)
cv2.imshow("Detected Text Regions", cv2.resize(image_with_boxes, (800, 800)))  # Scale down for better display

# Display Cropped Regions
for idx, cropped in cropped_regions:
    if cropped.size > 0:  # Ensure the cropped region is valid
        cv2.imshow(f"Cropped Region {idx}", cropped)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np
from paddleocr import PaddleOCR

# Initialize PaddleOCR for DBNet text detection
ocr = PaddleOCR(det_db_box_thresh=0.3, det_db_unclip_ratio=2.0, use_gpu=False)

# Function to draw bounding boxes
def draw_detected_boxes(image, result):
    for line in result[0]:
        print(f"Line: {line}")  # Debugging: Inspect each line in result
        box = line[0]  # Extract box coordinates
        score = line[1]  # Extract confidence score

        # Ensure score is a float
        if isinstance(score, list):
            score = score[0]  # Access the first element if it's a list

        pts = np.array(box, dtype=np.int32).reshape((-1, 2))
        # Draw the bounding box
        cv2.polylines(image, [pts], isClosed=True, color=(0, 255, 255), thickness=2)
        # Display confidence score
        x, y = pts[0]
        cv2.putText(image, f"{float(score):.2f}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return image

# Load image
image_path = r"D:\jersey_testing\2024-JUNE 13-MONDAY-TIME 1-FIELD 1-324-GAME-26475-TOURNAMENT-88.JPG"
image = cv2.imread(image_path)
if image is None:
    print("Error: Image could not be loaded. Please check the path.")
    exit()

# Run text detection
result = ocr.ocr(image, det=True, rec=False)

# Debugging: Print result structure
print(result[0])  # Inspect detected boxes and confidence scores

# Draw detected text boxes
image_with_boxes = draw_detected_boxes(image.copy(), result)

# Display results
h, w, _ = image_with_boxes.shape
scale = 800 / max(h, w)
resized_image = cv2.resize(image_with_boxes, (int(w * scale), int(h * scale)))

cv2.imshow("Detected Text Regions", resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


[2025/01/03 14:51:15] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\harsh.n/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.3, det_db_unclip_ratio=2.0, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\harsh.n/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_

In [4]:
import cv2
import numpy as np
from paddleocr import PaddleOCR

# Initialize PaddleOCR for DBNet text detection
ocr = PaddleOCR(det_db_box_thresh=0.3, det_db_unclip_ratio=1.5, use_gpu=False)

# Preprocess Image Function
def preprocess_image(image, target_size=(640, 640)):
    original_h, original_w, _ = image.shape
    scale = min(target_size[0] / original_h, target_size[1] / original_w)
    new_h, new_w = int(original_h * scale), int(original_w * scale)

    resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    # Add padding
    padded_image = np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)
    padded_image[:new_h, :new_w] = resized_image

    return padded_image, scale, (new_h, new_w)

# Draw Detected Boxes
def draw_detected_boxes(image, result, scale, resized_shape, target_size):
    original_h, original_w = image.shape[:2]
    resized_h, resized_w = resized_shape

    for line in result[0]:
        box = line[0]  # Extract box coordinates
        score = line[1]  # Extract confidence score

        # Map the box coordinates back to the original image
        box = np.array(box, dtype=np.float32)
        box[:, 0] = box[:, 0] * (resized_w / target_size[1])
        box[:, 1] = box[:, 1] * (resized_h / target_size[0])
        box[:, 0] /= scale
        box[:, 1] /= scale

        # Draw the bounding box
        pts = np.array(box, dtype=np.int32).reshape((-1, 2))
        cv2.polylines(image, [pts], isClosed=True, color=(0, 255, 255), thickness=2)
        # Display confidence score
        x, y = pts[0]
        cv2.putText(image, f"{score:.2f}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    return image

# Load image
image_path = r"D:\jersey_testing\your_image.jpg"
image = cv2.imread(image_path)
if image is None:
    print("Error: Image could not be loaded. Please check the path.")
    exit()

# Preprocess image
target_size = (640, 640)
preprocessed_image, scale, resized_shape = preprocess_image(image, target_size)

# Run text detection
result = ocr.ocr(preprocessed_image, det=True, rec=False)

# Draw detected text boxes
image_with_boxes = draw_detected_boxes(image.copy(), result, scale, resized_shape, target_size)

# Display results
h, w, _ = image_with_boxes.shape
resized_output = cv2.resize(image_with_boxes, (800, int(800 * h / w)))

cv2.imshow("Detected Text Regions", resized_output)
cv2.waitKey(0)
cv2.destroyAllWindows()


[2025/01/03 14:54:15] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\harsh.n/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.3, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\harsh.n/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_

AttributeError: 'NoneType' object has no attribute 'shape'

: 